In [8]:
import glob
import cv2
import numpy as np
import time
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Activation
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split

In [10]:
# Define path and categories
path = '/Users/tessapino/Downloads/lung_cancer_detection/dataset_kaggle'
categories = ['lung_aca', 'lung_n', 'lung_scc']

# Load dataset and labels
dataset = []
label = []

for i, cat in enumerate(categories):
    scans = glob.glob(f'{path}/{cat}/*.jpeg')
    for scan in scans:
        z = cv2.imread(scan)
        dataset.append(cv2.resize(z, (256, 256)))
        label.append(i)

# Convert to numpy arrays
dataset = np.asarray(dataset)
label = np.asarray(label)

In [11]:
# Split into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(dataset, label, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [12]:
# Build our model
dense_layers = [2]
layer_sizes = [64]
conv_layers = [2, 3]

In [13]:
for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
            print(NAME)

            model = Sequential()

            model.add(Conv2D(layer_size, (3, 3), input_shape=(256, 256, 3)))
            model.add(Activation('relu'))
            model.add(MaxPooling2D(pool_size=(2, 2)))

            for l in range(conv_layer-1):
                model.add(Conv2D(layer_size, (3, 3)))
                model.add(Activation('relu'))
                model.add(MaxPooling2D(pool_size=(2, 2)))

            model.add(Flatten())

            for _ in range(dense_layer):
                model.add(Dense(layer_size))
                model.add(Activation('relu'))

            model.add(Dense(1))
            model.add(Activation('sigmoid'))

            tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

            model.compile(loss='binary_crossentropy',
                          optimizer='adam',
                          metrics=['accuracy'])

            es = EarlyStopping(monitor='val_accuracy', min_delta=0.01, patience=5, verbose=1, mode='auto')
            mc = ModelCheckpoint(monitor='val_accuracy', filepath='./bestmodelmask.h5', verbose=1, save_best_only=True, mode='auto')
            tb = TensorBoard(log_dir="logs/{}".format(NAME))
            
            cb = [es, mc, tb]

            model.fit(X_train, y_train,
                      batch_size=32,
                      epochs=25,
                      validation_data=(X_val, y_val),
                      callbacks=cb)

2-conv-64-nodes-2-dense-1716984038
Epoch 1/25
282/282 [==============================] - ETA: 0s - loss: -3544.9634 - accuracy: 0.3301
Epoch 1: val_accuracy improved from -inf to 0.33500, saving model to ./bestmodelmask.h5
282/282 [==============================] - 676s 2s/step - loss: -3544.9634 - accuracy: 0.3301 - val_loss: -38907.9297 - val_accuracy: 0.3350
Epoch 2/25
282/282 [==============================] - ETA: 0s - loss: -17662134.0000 - accuracy: 0.3116
Epoch 2: val_accuracy did not improve from 0.33500
282/282 [==============================] - 679s 2s/step - loss: -17662134.0000 - accuracy: 0.3116 - val_loss: -106209672.0000 - val_accuracy: 0.3350
Epoch 3/25
282/282 [==============================] - ETA: 0s - loss: -820396544.0000 - accuracy: 0.3003
Epoch 3: val_accuracy improved from 0.33500 to 0.33933, saving model to ./bestmodelmask.h5
282/282 [==============================] - 732s 3s/step - loss: -820396544.0000 - accuracy: 0.3003 - val_loss: -2388472064.0000 - val_ac

KeyboardInterrupt: 

In [ ]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_acc}")